In [2]:
import os

In [3]:
%pwd

'c:\\Projects\\End-to-End_Cancer_Classification_using_DVC_MLFLOW\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Projects\\End-to-End_Cancer_Classification_using_DVC_MLFLOW'

In [ ]:
import dagshub
dagshub.init(repo_owner='harsh9769', repo_name='End-to-End_Cancer_Classification_using_DVC_MLFLOW', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

In [ ]:
https://dagshub.com/harsh9769/End-to-End_Cancer_Classification_using_DVC_MLFLOW.mlflow

In [12]:
from pathlib import Path

In [9]:
import tensorflow as tf

In [10]:
model=tf.keras.models.load_model("artifacts/training/model.h5")

In [17]:
from dataclasses import dataclass
@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [16]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories,save_json

In [18]:
class ConfigurationManager():
    def __init__(
        self,
        config_file_path= CONFIG_FILE_PATH,
        params_file_path= PARAMS_FILE_PATH):

        self.config= read_yaml(config_file_path)
        self.params= read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self)-> EvaluationConfig:

        eval_config=EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/harsh9769/End-to-End_Cancer_Classification_using_DVC_MLFLOW.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config
    
    

In [19]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [26]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1. / 255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=tuple(self.config.params_image_size[:2]),
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        # Set the MLflow registry URI
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        # Initialize DagsHub integration
        dagshub.init(
            repo_owner='harsh9769',
            repo_name='End-to-End_Cancer_Classification_using_DVC_MLFLOW',
            mlflow=True
        )

        with mlflow.start_run():
            # Log parameters and metrics
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({"loss": self.score[0], "accuracy": self.score[1]})
            mlflow.log_param('additional_param', 'value')
            mlflow.log_metric('additional_metric', 1)

            # Check if tracking URL type is not a file store
            if tracking_url_type_store != "file":
                # Log and register the model in MLflow's model registry
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                # Log the model locally
                mlflow.keras.log_model(self.model, "model")


In [27]:
import dagshub

In [28]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-12-25 15:16:19,361: INFO: common: yaml file:config\config.yaml loaded successfully]
[2024-12-25 15:16:19,365: INFO: common: yaml file:params.yaml loaded successfully]
[2024-12-25 15:16:19,366: INFO: common: created directory:artifacts]
Found 102 images belonging to 2 classes.
7/7 [==============================] - 7s 901ms/step - loss: 0.7999 - accuracy: 0.8039
[2024-12-25 15:16:26,542: INFO: common: json file saved at scores.json]
[2024-12-25 15:16:27,289: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Accessing as harsh9769

[2024-12-25 15:16:27,305: INFO: helpers: Accessing as harsh9769]
[2024-12-25 15:16:28,505: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/harsh9769/End-to-End_Cancer_Classification_using_DVC_MLFLOW "HTTP/1.1 200 OK"]
[2024-12-25 15:16:28,992: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "harsh9769/End-to-End_Cancer_Classification_using_DVC_MLFLOW"

[2024-12-25 15:16:28,997: INFO: helpers: Initialized MLflow to track repo "harsh9769/End-to-End_Cancer_Classification_using_DVC_MLFLOW"]


Repository harsh9769/End-to-End_Cancer_Classification_using_DVC_MLFLOW initialized!

[2024-12-25 15:16:29,000: INFO: helpers: Repository harsh9769/End-to-End_Cancer_Classification_using_DVC_MLFLOW initialized!]


2024/12/25 15:16:31 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-12-25 15:16:32,092: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\Harsh\AppData\Local\Temp\tmpllzj4xkv\model\data\model\assets
[2024-12-25 15:16:32,545: INFO: builder_impl: Assets written to: C:\Users\Harsh\AppData\Local\Temp\tmpllzj4xkv\model\data\model\assets]
